<img src='img/logo.png'>
<img src='img/title.png'>

# Clustering unlabeld data

Clustering refers to the collection of algorithms that search for regions of similarity in a collection of unlabeled data.

The goal of cluster algorithms is to divide a collection of points into discrete labels.

Each method makes particular assumptions about the expected shapes and densities of the clusters to be modeled.

# Table of Contents
* [Clustering unlabeld data](#Clustering-unlabeld-data)
* [Setup](#Setup)
* [Blobs, moons and circles](#Blobs,-moons-and-circles)
* [KMeans](#KMeans)
	* [Fitting](#Fitting)
	* [Labels](#Labels)
	* [Incorrect assumptions](#Incorrect-assumptions)
* [Agglomerative Clustering](#Agglomerative-Clustering)
	* [Complex cluster shapes](#Complex-cluster-shapes)
* [Density-based clustering](#Density-based-clustering)
	* [Labels and noise](#Labels-and-noise)
	* [Complex shapes](#Complex-shapes)


# Setup

In [ ]:
#Fix working directory
%cd notebooks

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from matplotlib.cm import Accent
%matplotlib inline

import hvplot.pandas
import holoviews as hv
import panel as pn
hv.extension('bokeh')

import src.mglearn as mglearn

# Blobs, moons and circles

In this notebook we'll start with randomly generated datasets in two dimensions to demonstrate the clustering algorithms.

Since we're doing *unsupervised* learning the labels will be used for validation against the entire dataset.

In [ ]:
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs, make_moons, make_circles

In [ ]:
blobs, blobs_y = make_blobs(n_features=2, n_samples=1000, centers=2, random_state=0)
moons, moons_y = make_moons(n_samples=1000, noise=0.05, random_state=0)
circles, circles_y = make_circles(n_samples=1000, noise=0.05, factor=0.5)

In [ ]:
plot_options = dict(size=8, width=450, height=300, color='Category', cmap='Accent', padding=0.1)

b = hv.Points((blobs[:,0], blobs[:,1], blobs_y), vdims=['Category'], label='Blobs').options(**plot_options)
m = hv.Points((moons[:,0], moons[:,1], moons_y), vdims=['Category'], label='Moons').options(**plot_options)
c = hv.Points((circles[:,0], circles[:,1], circles_y), vdims=['Category'], label='Circles').options(**plot_options)

pn.Column(pn.Row(c,m), b)

# KMeans

The KMeans algorithm seeks to divide the input data into `n_clusters` by assigning points to one of the centers. The center positions are optimized to minimize the distance from each point to the center.

Upon convergence the cluster centers are averages of the points to which they are assigned.

The KMeans clustering algorithm *requires* that the expected number of labels be provided.

In [ ]:
from sklearn.cluster import KMeans

## Fitting

In [ ]:
kmeans = KMeans(n_clusters=2)

kmeans.fit(blobs)

## Labels

The `label_` attribute is the label assignment for the input data.

In [ ]:
kmeans.labels_[:10]

The `.predict()` method can predict labels for new data.

In [ ]:
kmeans.predict(blobs)[:10]

KMeans assumes that each cluster has the same diameter.


The boundary is a straight line between the the two centers (red points).

In [ ]:
b + \
(hv.Points((blobs[:,0], blobs[:, 1], kmeans.predict(blobs)),
              label='KMeans', vdims=['Category']).options(**plot_options) \
* hv.Points(kmeans.cluster_centers_))

## Incorrect assumptions

KMeans assumes that the clusters are *convex and isotropic* and do not overlap.

Preprocessing a large feature space with [Principle Component Analysis](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) to reduce the number of dimensions is recommended.

In [ ]:
kmeans.fit(moons)

In [ ]:
m + \
(hv.Points((moons[:,0], moons[:, 1], kmeans.predict(moons)),
              label='KMeans', vdims=['Category']).options(**plot_options) \
* hv.Points(kmeans.cluster_centers_))

In [ ]:
kmeans.fit(circles)

c + \
(hv.Points((circles[:,0], circles[:, 1], kmeans.predict(circles)),
              label='KMeans', vdims=['Category']).options(**plot_options) \
* hv.Points(kmeans.cluster_centers_))

By requesting many clusters it may actually be possible to separate the two moons.

Notice that the new labels may be further modeled using a supervised learning method like KNeighbors.

In [ ]:
kmeans = KMeans(n_clusters=10)

kmeans.fit(moons)

In [ ]:
m + \
(hv.Points((moons[:,0], moons[:, 1], kmeans.predict(moons)),
              label='KMeans', vdims=['Category']).options(**plot_options) \
* hv.Points(kmeans.cluster_centers_))

# Agglomerative Clustering

Agglomerative clustering *grows* clusters from each point in the data set.

Starting with every point in its own cluster, the clusters are merged by proximity until `n_clusters` have been generated.

In [ ]:
from sklearn.cluster import AgglomerativeClustering

In [ ]:
agg = AgglomerativeClustering(n_clusters=2)

In [ ]:
agg.fit(blobs)

In [ ]:
b + hv.Points((blobs[:,0], blobs[:, 1], agg.labels_),
              label='Agglomerative', vdims=['Category']).options(**plot_options)

## Complex cluster shapes

AgglomerativeClustering is still having trouble with moons and circles.

In [ ]:
agg.fit(moons)

m + hv.Points((moons[:,0], moons[:, 1], agg.labels_),
              label='Agglomerative', vdims=['Category']).options(**plot_options)

In [ ]:
agg.fit(circles)

c + hv.Points((circles[:,0], circles[:, 1], agg.labels_),
              label='Agglomerative', vdims=['Category']).options(**plot_options)

# Density-based clustering

The DBSCAN model assumes that clusters form dense regions in space *separated* by relatively sparse regions.

DBSCAN does not need to know the expected number of clusters, but will discover the clusters iteratively as regions of high density.

Regions of high density are defined as
* number of points `min_samples`
* within a distance of `eps` from any point

All other points are defined as noise.

Small values of `eps` mean that all points will be labeled as noise.

Large values of `eps` assign all points to a single cluster.

In [ ]:
from sklearn.cluster import DBSCAN

In [ ]:
dbscan = DBSCAN()

In [ ]:
dbscan.fit(blobs)

## Labels and noise

For the blobs DBSCAN only found one real cluster.

Points assigned to `-1` are considered *noise* since they do not belong to the cluster.

In [ ]:
np.unique(dbscan.labels_, return_counts=True)

In [ ]:
dbscan.labels_[-25:]

Most of the points are labeled in a single cluster with a few noise points.

In [ ]:
b + hv.Points((blobs[:,0], blobs[:, 1], dbscan.labels_),
              label='DBSCAN', vdims=['Category']).options(**plot_options)

## Complex shapes

With the right value of `eps` the moons clusters can be modeled.

It is important to normalize the data first with the StandardScaler or MinMaxScaler to normalize all features to similar ranges.

In [ ]:
dbscan = DBSCAN(eps=0.2)

dbscan.fit(moons)

In [ ]:
m + hv.Points((moons[:,0], moons[:, 1], dbscan.labels_),
              label='DBSCAN', vdims=['Category']).options(**plot_options)

In [ ]:
dbscan = DBSCAN(eps=0.2)

dbscan.fit(circles)

In [ ]:
c + hv.Points((circles[:,0], circles[:, 1], dbscan.labels_),
              label='DBSCAN', vdims=['Category']).options(**plot_options)

<img src='img/copyright.png'>